In [3]:
import mysql.connector
from datetime import datetime

In [5]:

# MySQL database connection
db_config = {
    "host": "localhost",  # Replace with your MySQL server address
    "user": "root",       # Replace with your MySQL username
    "password": "deva@T555"  # Replace with your MySQL password
}

# Connect to MySQL server
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Create the database if it does not exist
cursor.execute("CREATE DATABASE IF NOT EXISTS BillDBMS")
cursor.execute("USE BillDBMS")

In [6]:
# Define table structures
table_structure = {
    'bill_info': '''
        CREATE TABLE IF NOT EXISTS bill_info (
            BillID INT AUTO_INCREMENT PRIMARY KEY,
            StoreName VARCHAR(255) NOT NULL,
            StoreAddress TEXT NOT NULL,
            Tel VARCHAR(20) NOT NULL,
            BillDate DATE NOT NULL,
            BillTime TIME NOT NULL,
            SubTotal DECIMAL(10, 2) NOT NULL,
            Total DECIMAL(10, 2) NOT NULL,
            Tax DECIMAL(10, 2) NOT NULL,
            Tips DECIMAL(10, 2)
        );
    ''',
    'products': '''
        CREATE TABLE IF NOT EXISTS products (
            ProductID INT AUTO_INCREMENT PRIMARY KEY,
            BillID INT,
            ProductName VARCHAR(255) NOT NULL,
            Quantity INT NOT NULL,
            Price DECIMAL(10, 2) NOT NULL,
            FOREIGN KEY (BillID) REFERENCES bill_info (BillID)
        );
    ''',
    'images': '''
        CREATE TABLE IF NOT EXISTS images (
            id INT AUTO_INCREMENT PRIMARY KEY,
            bill_img LONGBLOB NOT NULL,
            upload_date DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
            BillID INT,
            FOREIGN KEY (BillID) REFERENCES bill_info (BillID)
        );
    '''
}

# Create tables
for table_name, create_query in table_structure.items():
    cursor.execute(create_query)
print("Tables created successfully.")


Tables created successfully.


In [7]:
# Insert sample data into bill_info
cursor.execute('''
    INSERT INTO bill_info (StoreName, StoreAddress, Tel, BillDate, BillTime, SubTotal, Total, Tax, Tips)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
''', ("SuperMart", "123 Main St, City", "123-456-7890", "2024-11-17", "14:30:00", 90.0, 100.0, 10.0, 5.0))

# Get the BillID of the inserted row
bill_id = cursor.lastrowid

# Insert sample data into products
products = [
    (bill_id, "Apples", 2, 3.0),
    (bill_id, "Bread", 1, 2.5),
    (bill_id, "Milk", 1, 1.5)
]
cursor.executemany('''
    INSERT INTO products (BillID, ProductName, Quantity, Price)
    VALUES (%s, %s, %s, %s)
''', products)

# Insert sample data into images
with open("sample_bill.jpg", "rb") as f:  # Replace with a path to an actual image file
    image_blob = f.read()
cursor.execute('''
    INSERT INTO images (bill_img, BillID)
    VALUES (%s, %s)
''', (image_blob, bill_id))

# Commit changes and close the connection
conn.commit()
print("Sample data inserted successfully.")

# Close the database connection
cursor.close()
conn.close()

Sample data inserted successfully.
